# Modeling the Ribosome in COBRAme

## 1. Load the python packages

In [1]:
from __future__ import print_function, division, absolute_import

# python imports
import re
from os.path import join
from collections import defaultdict
import pickle
import pandas as pd

# third party imports
import pandas
import tabulate
import cobra
pd.set_option('display.max_columns', 100)
pd.set_option('display.width',100)
pd.set_option('display.max_colwidth',100)

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
#from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/solvers/__init__.py:63: UserWarning: No LP solvers found
  warn("No LP solvers found")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


## 2. Load the iJL1678b model

Load the iJL1678b model in pickle format that is located in the current directory.

In [2]:
with open('iJL1678b.pickle', 'rb') as f:
    me = pickle.load(f)

## 3. Finding the reactions that produce the ribosomes (ComplexFormation)

Let's begin by finding the reactants used to create the ribosome fomr the "Complex" reaction "formation_ribosome."

In [3]:
me.reactions.get_by_id('formation_ribosome').reactants

[<Complex RpsD_mono at 0x7f1dcacdd4e0>,
 <Complex RpsE_mono at 0x7f1dc621b438>,
 <Complex RpsF_mono at 0x7f1dc646b8d0>,
 <Complex RpsG_mono at 0x7f1dc62847f0>,
 <Complex RpsH_mono at 0x7f1dc62a7d30>,
 <Complex RpsI_mono at 0x7f1dc641ada0>,
 <Complex RpsK_mono at 0x7f1dc63e94a8>,
 <Complex RpsL_mono at 0x7f1dc638d278>,
 <Complex RpsM_mono at 0x7f1dc620e2e8>,
 <Complex RpsO_mono at 0x7f1dc6352d30>,
 <Complex RpsP_mono at 0x7f1dc6460550>,
 <Complex RpsQ_mono at 0x7f1dc61bf198>,
 <Complex RpsR_mono at 0x7f1dc639a358>,
 <Complex RpsS_mono at 0x7f1dc633c320>,
 <Complex RpsT_mono at 0x7f1dc6319470>,
 <GenericComponent generic_16s_rRNAs at 0x7f1dc48e6c18>,
 <Complex RpsA_mono at 0x7f1dc628e780>,
 <Complex RpsB_mono at 0x7f1dc63474e0>,
 <Complex RpsC_mono at 0x7f1dc620ec88>,
 <Complex RpsJ_mono at 0x7f1dcacdd4a8>,
 <Complex RpsN_mono at 0x7f1dc63d3e48>,
 <Complex RpsU_mono at 0x7f1dc6319ef0>,
 <Complex Sra_mono at 0x7f1dc61bff98>,
 <GenericComponent generic_23s_rRNAs at 0x7f1dc48e6f98>,
 <Gener

Reorganizing these reactants.

**30S ribosomal subunit**
1. 16S ribosomal RNA = RrsA (b3851, rrsA, TUO-1181)
2. 30S ribosomal subunit protein S1 = RpsA (b0911, rpsA, TU361/TU564/TU565)
3. 30S ribosomal subunit protein S2 = RpsB (b0169, rpsB, TU00345)
4. 30S ribosomal subunit protein S3 = RpsC (b3314, rpsC, TU00347)
5. 30S ribosomal subunit protein S4 = RpsD (b3296, rpsD,TU00349/TU0-14658)
6. 30S ribosomal subunit protein S5 = RpsE (b3303, rpsE, TU00337)
7. 30S ribosomal subunit protein S6 = RpsF (b4200, rpsF, TU00346)
8. 30S ribosomal subunit protein S7 = RpsG (b3341, rpsG, TU00348)
9. 30S ribosomal subunit protein S8 = RpsH (b3306, rpsH, TU00337)
10. 30S ribosomal subunit protein S9 = RpsI (b3230, rpsI, TU00336)
11. 30S ribosomal subunit protein S10 = RpsJ (b3321, rpsJ, TU00347)
12. 30S ribosomal subunit protein S11 = RpsK (b3297, rpsK, TU00349/TU0-14658)
13. 30S ribosomal subunit protein S12 = RpsL (b3329, rpsL, TU00348)
14. 30S ribosomal subunit protein S13 = RpsM (b3298, rpsM, TU00349/TU0-14658)
15. 30S ribosomal subunit protein S14 = RpsN (b3307, rpsN, TU00347)
16. 30S ribosomal subunit protein S15 = RpsO (b3165, rpsO, TU341/TU0-8474)
17. 30S ribosomal subunit protein S16 = RpsP (b2609, rpsP, TU00351)
18. 30S ribosomal subunit protein S17 = RpsQ (b3311, rpsQ, TU00347)
19. 30S ribosomal subunit protein S18 = RpsR (b4202, rpsR, TU00346)
20. 30S ribosomal subunit protein S19 = RpsS (b3316, rpsS, TU00347)
21. 30S ribosomal subunit protein S20 = RpsT (b0023, rpsT, TU0-7842/TU0-7844)
22. 30S ribosomal subunit protein S21 = RpsU (b3065, rpsU, TU00352/TU00434/TU00435)

**50S ribosomal subunit**
1. 23S ribosomal RNA = RrlA (b3854, rrlA, TU0-1181/TU0-45165/TU0-14486)
2. 5S ribosomal RNA = RrfA (b3855, rrfA, TU0-1181/TU0-45165/TU0-14486)
3. 50S ribosomal subunit protein L1 = RplA (b3984, rplA, TU0-6512)
4. 50S ribosomal subunit protein L2 = RplB (b3317, rplB, TU00347)
5. 50S ribosomal subunit protein L3 = RplC (b3320, rplC, TU00347)
6. 50S ribosomal subunit protein L4 = RplD (b3319, rplD, TU00347)
7. 50S ribosomal subunit protein L5 = RplE (b3308, rplE, TU00337)
8. 50S ribosomal subunit protein L6 = RplF (b3305, rplF, TU00337)
9. 50S ribosomal subunit protein L7/L12 dimer = (RplL) (b3986, rplL, TU0-6512/TU00335/TU0-8464)
10. 50S ribosomal protein complex L8 = (RplJ)([RplL]) (b3985, rplJ, TU0-6512/TU00335 AND b3986, rplL, TU0-6512/TU00335/TU0-8464)
11. 50S ribosomal subunit protein L9 = RplI (b4203, rpll, TU00346)
12. 50S ribosomal subunit protein L10 = RplJ (b3985, rplJ, TU0-6512/TU00335)
13. 50S ribosomal subunit protein L11 = RplK (b3983, rplL, TU0-6512)
14. 50S ribosomal subunit protein L12 = RplL (b3986, rplL, TU0-6512/TU00335/TU0-8464)
15. 50S ribosomal subunit protein L13 = RplM (b3231,rplM, TU00336)
16. 50S ribosomal subunit protein L14 = RplN (b3310, rplN, TU00337)
17. 50S ribosomal subunit protein L15 = RplO (b3301, rplO, TU00337)
18. 50S ribosomal subunit protein L16 = RplP (b3313, rplP, TU00347)
19. 50S ribosomal subunit protein L17 = RplQ (b3294, rplQ, TU00349 or TU0-14658 )
20. 50S ribosomal subunit protein L18 = RplR (b3304, rplR, TU00337)
21. 50S ribosomal subunit protein L19 = RplS (b2606, rplS, TU00351)
22. 50S ribosomal subunit protein L20 = RplT (b1716, rplT, TU0-14257/TU0-3305/TU0-14255/TU0-8855)
23. 50S ribosomal subunit protein L21 = RplU (b3186,rplU, TU0-42628)
24. 50S ribosomal subunit protein L22 = RplV (b3315, rplV, TU00347)
25. 50S ribosomal subunit protein L23 = RplW (b3318, rplW, TU00347)
26. 50S ribosomal subunit protein L24 = RplX (b3309, rplX, TU00337)
27. 50S ribosomal subunit protein L25 = RplY (b2185, rplY, TU0-13557/PM0-45275/PM0-45274/PM0-45273)
28. 50S ribosomal subunit protein L27 = RpmA (b3185, rpmA, TU0-42628)
29. 50S ribosomal subunit protein L28 = RpmB (b3637, rpmB, TU0-5522/TU0-5543)
30. 50S ribosomal subunit protein L29 = RpmC (b3312, rpmC, TU00347)
31. 50S ribosomal subunit protein L30 = RpmD (b3302, rpmD, TU00337)
32. 50S ribosomal subunit protein L31 = RpmE (b3936, rmpE, TU0-45156/TU0-8506)
33. 50S ribosomal subunit protein L32 = RpmF (b1089, rpmF, TU0-1/TU601/TU602)
34. 50S ribosomal subunit protein L33 = RpmG (b3636, rpmG, TU0-5522/TU0-5543)
35. 50S ribosomal subunit protein L34 = RpmH (b3703,rpmH, TU743/TU00340/TU682)
36. 50S ribosomal subunit protein L35 = RpmI (b1717, rpmI, TU0-14257/TU0-3305/TU0-14255)
27. 50S ribosomal subunit protein L36 = RpmJ (b3299, rpmJ, TU00337)

**16S rRNA methyltransferases**
1. KsgA_mono, 16S rRNA dimethyltransferase (b0051, rmsa, TU00362/TU0-5221/TU0-5186)
2. RsmB_mono, 16S rRNA methyltransferase (b3289, rsmB, TU0-14355) 
3. RsmC_mono, 16S rRNA methyltransferase (b4371, rsmC, TU0-14215/PM0-46104/PM0-46105)
4. RsmD_mono, 16S rRNA methyltransferase (b3465, rsmD, TU0-13942/PM0-45784/PM0-46554/PM0-45785/PM0-45783)
5. YggJ_dim (rsmE), 16S rRNA methyltransferase (b2946, rsmE, TU0-13805/PM0-45543/PM0-45544)
6. RsmF_mono, 16S rRNA methyltransferase (b1835, rsmF, TU0-13428) 
7. RsmG_mono, 16S rRNA methyltransferase (b3740, rsmG, TU0-45168/TU0-14029)

**23S rRNA methyltransferases**
1. RrmA_dim_mod_2:zn2, 23S rRNA methyltransferase (b1822, rlmA, TU0-13421/PM0-45104)
2. RlmB_dim, 23S rRNA methyltransferase (b4180, rlmB, TU0-8502)
3. RumB_mono_mod_1:4fe4s, 23S rRNA methyltransferase (b0859, rlmC, TU0-13072/PM0-44652/PM0-44653)
4. RumA_mono_mod_1:4fe4s, 23S rRNA methyltransferase (b2785, rlmD, TU0-13735)
5. RrmJ_mono, 23S rRNA methyltransferase (b3179, rlmE, TU00277/TU00414)
6. RlmF_mono, 23S rRNA methyltransferase (b0807, rlmF, TU0-8568/PM0-44630/PM0-44631)
7. RlmG_mono, 23S rRNA methyltransferase (b3084, rlmG, TU0-13855/PM0-45623)
8. RlmH_dim, 23S rRNA methyltransferase (b0636, rlmH, TU0-13009)
9. RlmI_dim, 23S rRNA methyltransferase (b0967, rlmI, TU0-13103/PM0-44694)
10. RlmL_dim, fused 23S rRNA methyltransferase (b0948, rlmL, TU0-2001)
11. RlmM_mono, 23S rRNA methyltransferase (b2806, rlmM, TU0-14306/PM0-45481)
12. RlmN_mono_mod_1:4fe4s, 23S rRNA methyltransferase/tRNA methyltransferase (b2517, rlmN, TU0-13650)

**23S rRNA pseudouridine synthase**
1. RluA_mono, 23S rRNA pseudouridine and tRNA pseudouridine synthase (b0058, rluA, TU0-3841)
2. RluB_mono, 23S rRNA pseudouridine synthase (b1269, rluB, TU0-13195)
3. RluC_mono, 23S rRNA pseudouridine synthase (b1086, rluC, TU0-13134)
4. RluD_mono_mod_1:mg2, 23S rRNA pseudouridine synthase (b2594, rluD, TU0-13675)
5. YmfC_mono, 23S rRNA pseudouridine synthase (b1135, rluE, TU0-13145)
6. YjbC_mono (rluF), 23S rRNA pseudouridine and tRNA-Tyr pseudouridine synthase (b4022, rluF, TU0-8505/PM0-45984/PM0-45985)

**16S rRNA pseudouridine synthase**
1. RsuA_mono, 16S rRNA pseudouridine synthase (b2183, rsuA, TU0-13555/PM0-45272/PM0-45271)

**30S ribosomal maturation**
1. Era_dim, 30S ribosomal subunit maturation GTPase Era (b2566, era, TU483)
2. RimM_mono, ribosome maturation factor RimM (b2608, rimM, TU00351)
3. Tig_mono, chaperone protein; trigger factor (b0436, tig, TU0-12928/PM0-9326/TU0-14614)
4. Sra_mono, ribosome-associated protein Sra (b1480, sra, TU0-7141/TU0-3901)

**30S ribosome binding factor**
1. RbfA_mono, 30S ribosome binding factor (b3167, rbfA, TU341)

**Metabolites**
1. gtp_c
2. amet_c
3. nadh_c
4. h2o_c
5. mg2_c

https://www.biocyc.org/ECOLI/NEW-IMAGE?type=ENZYME&object=CPLX0-3964

The products of the "formation_ribosome" reaction are

In [4]:
me.reactions.get_by_id('formation_ribosome').products

[<Constraint prosthetic_group_biomass at 0x7f1dcadd63c8>,
 <Ribosome ribosome at 0x7f1dc9fc34e0>,
 <Metabolite ahcys_c at 0x7f1dc67ad400>,
 <Metabolite h_c at 0x7f1dcacb6cf8>,
 <Metabolite nad_c at 0x7f1dc619bfd0>,
 <Metabolite pi_c at 0x7f1dcacb6c18>,
 <Metabolite gdp_c at 0x7f1dc9fc3b00>]

The total formation_ribosome reaction is

In [5]:
me.reactions.get_by_id('formation_ribosome').reaction

'4.27350427350427e-6*mu Era_dim + 8.54700854700855e-6*mu KsgA_mono + 4.27350427350427e-6*mu RbfA_mono + 4.27350427350427e-6*mu RimM_mono + 4.27350427350427e-6*mu RlmB_dim + 4.27350427350427e-6*mu RlmF_mono + 4.27350427350427e-6*mu RlmG_mono + 4.27350427350427e-6*mu RlmH_dim + 4.27350427350427e-6*mu RlmI_dim + 8.54700854700855e-6*mu RlmL_dim + 4.27350427350427e-6*mu RlmM_mono + 4.27350427350427e-6*mu RlmN_mono_mod_1:4fe4s + 4.27350427350427e-6*mu RluA_mono + 4.27350427350427e-6*mu RluB_mono + 1.28205128205128e-5*mu RluC_mono + 1.28205128205128e-5*mu RluD_mono_mod_1:mg2 + RplA_mono + RplB_mono + RplC_mono + RplD_mono + RplE_mono + RplF_mono + RplI_mono + RplJ_mono + RplK_mono + RplM_mono + RplN_mono + RplO_mono + RplP_mono + RplQ_mono + RplR_mono + RplS_mono + RplT_mono + RplU_mono + RplV_mono + RplW_mono + RplX_mono + RplY_mono + RpmA_mono + RpmB_mono + RpmC_mono + RpmD_mono + RpmE_mono + RpmF_mono + RpmG_mono + RpmH_mono + RpmI_mono + RpmJ_mono + RpsA_mono + RpsB_mono + RpsC_mono + Rps